# Download the Latest ERA5 Datasets Using the Climate Data Store (CDS)

The most up-to-date meteorological data used for running the JULES model are sourced from the [Climate Data Store (CDS)](https://cds-beta.climate.copernicus.eu/datasets/reanalysis-era5-single-levels) via the **CDS API**. This ensures that the simulations are based on high-quality reanalysis datasets with a consistent temporal and spatial resolution.

### Meteorological Forcing Variables for JULES

The following key meteorological forcing variables are required for running the **JULES** land surface model:

- **tas**: Near-surface air temperature (2 meters above ground)
- **tasrange**: Range between daily maximum and minimum temperature
- **pr**: Total precipitation
- **ps**: Surface pressure
- **sfcWind**: Near-surface wind speed
- **huss**: Near-surface specific humidity
- **rsds**: Surface downwelling shortwave radiation
- **rlds**: Surface downwelling longwave radiation

### Methodology

In total, 10 variables are retrieved from the **ERA5 single-level reanalysis** dataset at **hourly temporal resolution** and a horizontal resolution of **0.25° x 0.25°**. Some of these variables are used directly in the JULES model, meaninig that they only require  basic pre-processing, such as spatial aggregation and temporal remapping, to match the model's setup.

#### Calculations and Derived Variables:

Variables like tasrange, huss, hurs, and sfcwind had to be derived from other variables, as they are unavailable directly from ERA5.

- **tasrange**: The difference between daily maximum temperature (**tasmax**) and daily minimum temperature (**tasmin**).
- **huss** and **hurs**: Derived from near-surface dewpoint temperature (**dewptas**), surface pressure (**ps**), and near-surface air temperature (**tas**), using the equations described by **Buck (1981)** [[methodology](https://confluence.ecmwf.int/display/CKB/Near+surface+meteorological+variables+from+1979+to+2019+derived+from+bias-corrected+reanalysis+%28WFDE5%29%3A+Product+User+Guide#Nearsurfacemeteorologicalvariablesfrom1979to2019derivedfrombiascorrectedreanalysis(WFDE5):ProductUserGuide-references9)]. 
- **sfcWind**: Calculated from the eastward (**uas**) and northward (**vas**) near-surface wind components, using the formula:

\[
sfcWind = \sqrt{uas^2 + vas^2}
\]

### ERA5 Variables and Corresponding ISIMIP Naming Convention

Below is the list of the variables sourced from ERA5 and their corresponding ISIMIP naming conventions:

| ERA5 Variable Name                              | ISIMIP Variable Name | Usage/Calculation                          |
|-------------------------------------------------|----------------------|--------------------------------------------|
| 2m_temperature                                  | **tas**               | Used directly for near-surface temperature |
| maximum_2m_temperature_since_previous_post_processing | **tasmax**            | Used for calculating **tasrange**          |
| minimum_2m_temperature_since_previous_post_processing | **tasmin**            | Used for calculating **tasrange**          |
| 2m_dewpoint_temperature                         | **dewptas**           | Used for deriving **hurs** and **huss**    |
| total_precipitation                             | **pr**                | Used directly for precipitation            |
| surface_pressure                                | **ps**                | Used directly and for deriving **hurs** and **huss** |
| surface_solar_radiation_downwards               | **rsds**              | Used directly for shortwave radiation      |
| surface_thermal_radiation_downwards             | **rlds**              | Used directly for longwave radiation       |
| 10m_u_component_of_wind                         | **uas**               | Used for calculating **sfcWind**           |
| 10m_v_component_of_wind                         | **vas**               | Used for calculating **sfcWind**           |

### Notes:
- Variables such as **huss**, **hurs**, and **sfcWind** are not directly available in ERA5. They are calculated using other available ERA5 variables, following well-established equations.
- The **tasrange** variable is calculated as the difference between **tasmax** (maximum temperature) and **tasmin** (minimum temperature) to provide an estimate of daily temperature variability.



In [1]:
import cdsapi
import os
import time

In [2]:
# Function to download ERA5 data with retry mechanism
def download_era5_variable(download_dir, era5_var, isimip_var, years, retries=3, delay=300):
    """Download ERA5 data for a specific variable, with retries in case of failure.
    
    Parameters:
    - era5_var: The ERA5 variable name
    - isimip_var: The corresponding ISIMIP variable name (not used here, but could be useful for naming the output file)
    - years: List of years to download (provided as a list of strings)
    - retries: Number of retry attempts
    - delay: Delay between retries (in seconds)
    """
    
    # Initialize the CDS API client
    client = cdsapi.Client()

    # Define the full path for the output file
    output_file = os.path.join(download_dir, f"{era5_var}_hourly_{years[0]}_{years[-1]}.nc")

    # Define the ERA5 dataset for download
    dataset = "reanalysis-era5-single-levels"

    # Prepare the CDS API request
    request = {
        'product_type': ['reanalysis'],
        'variable': era5_var,
        'year': years,
        'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
        'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
        'time': ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00'],
        'data_format': 'netcdf',
        'download_format': 'unarchived'
    }

    attempt = 0
    while attempt < retries:
        try:
            print(f"Attempting to download {era5_var} [{isimip_var}] for {years}. Attempt {attempt + 1} of {retries}")
            client.retrieve(dataset, request).download(output_file)
            print(f"Download successful: {output_file}")
            break
        except Exception as e:
            attempt += 1
            print(f"Error downloading {era5_var}: {e}")
            if attempt < retries:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print(f"Failed to download {era5_var} after {retries} attempts.")
                return False
    return True


In [3]:
# Define the download directory
download_dir = 'V:/ERA5_data'

# Ensure the directory exists
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

# Define year of interest
years_to_download = ['2022', '2023', '2024']

# Dictionary of variables to download
var_dict = {'tas':'2m_temperature',
            'tasmin':'minimum_2m_temperature_since_previous_post_processing',
            'tasmax':'maximum_2m_temperature_since_previous_post_processing',
            'dewptas':'2m_dewpoint_temperature',
            'pr':'total_precipitation',
            'ps':'surface_pressure',
            'uas':'10m_u_component_of_wind',
            'vas':'10m_v_component_of_wind',
            'rsds':'surface_solar_radiation_downwards',
            'rlds':'surface_thermal_radiation_downwards',
}

In [5]:
# Main loop to download each variable
for isimip_var, era5_var in var_dict.items():
    success = download_era5_variable(download_dir, era5_var, isimip_var, years_to_download)
    if not success:
        print(f"Skipping {isimip_var} due to repeated failures.")
    else:
        print(f"Completed download for {isimip_var}.")

Attempting to download minimum_2m_temperature_since_previous_post_processing for ['2022', '2023', '2024']. Attempt 1 of 3


2024-09-06 17:15:59,432 INFO Request ID is f521f56d-2f78-4dbe-9ad6-d3cc2bce6b52
2024-09-06 17:15:59,532 INFO status has been updated to accepted
2024-09-06 17:50:19,847 INFO status has been updated to running
2024-09-06 20:10:31,653 INFO status has been updated to accepted
2024-09-06 20:12:31,796 INFO status has been updated to running
2024-09-06 20:26:32,866 INFO Creating download object as as_source with files:
['data_stream-oper.nc']
2024-09-06 22:02:40,579 INFO Creating download object as as_source with files:
['data_stream-oper.nc']
2024-09-06 22:34:43,176 INFO status has been updated to successful


Download successful: V:/ERA5_data\tasmin_hourly_2022_2024.nc
Completed download for tasmin.
Attempting to download maximum_2m_temperature_since_previous_post_processing for ['2022', '2023', '2024']. Attempt 1 of 3


2024-09-06 22:51:11,426 INFO Request ID is 60f3bdae-1315-432d-8556-7cbce7614ce5
2024-09-06 22:51:11,492 INFO status has been updated to accepted
2024-09-06 22:54:02,200 INFO status has been updated to running
2024-09-07 00:35:36,828 INFO Creating download object as as_source with files:
['data_stream-oper.nc']
2024-09-07 00:59:38,771 INFO status has been updated to successful


Download successful: V:/ERA5_data\tasmax_hourly_2022_2024.nc
Completed download for tasmax.
Attempting to download 2m_dewpoint_temperature for ['2022', '2023', '2024']. Attempt 1 of 3


...

Download successful: V:/ERA5_data\pr_hourly_2022_2024.nc
Completed download for pr.
Attempting to download surface_solar_radiation_downwards for ['2022', '2023', '2024']. Attempt 1 of 3


2024-09-07 07:10:35,990 INFO Request ID is be2eef4f-73fe-43ff-b3c1-ca11d9b4682d
2024-09-07 07:10:36,060 INFO status has been updated to accepted
2024-09-07 07:11:24,801 INFO status has been updated to running
2024-09-07 09:25:03,344 INFO Creating download object as as_source with files:
['data_stream-oper.nc']
2024-09-07 09:41:04,532 INFO status has been updated to successful


Download successful: V:/ERA5_data\rsds_hourly_2022_2024.nc
Completed download for rsds.
Attempting to download surface_thermal_radiation_downwards for ['2022', '2023', '2024']. Attempt 1 of 3


2024-09-07 10:10:10,736 INFO Request ID is 6c7b8982-672e-4105-ba89-e526ea894ee5
2024-09-07 10:10:10,803 INFO status has been updated to accepted
2024-09-07 10:10:12,401 INFO status has been updated to running
2024-09-07 12:34:38,461 INFO Creating download object as as_source with files:
['data_stream-oper.nc']
2024-09-07 13:10:41,163 INFO status has been updated to successful
                                                                                           

Download successful: V:/ERA5_data\rlds_hourly_2022_2024.nc
Completed download for rlds.
